In [1]:
# Checking host

import socket
print(socket.gethostname())

r45.uppmax.uu.se


In [2]:
import pandas as pd
import numpy as np

# Loading work from FindingPairs

AgeDB_df = pd.read_csv("createdCSV/AgeDB_pairs.csv", index_col=0)
CASIA_df = pd.read_csv("createdCSV/CASIA_pairs.csv", index_col=0)

In [3]:
AgeDB_df

,ID,Name,Gender,First Image,First Age,First Age Range,Second Image,Second Age,Second Age Range
0,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10044_HelenHunt_32_f.jpg,32,31-40
1,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10045_HelenHunt_33_f.jpg,33,31-40
2,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10046_HelenHunt_34_f.jpg,34,31-40
3,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10057_HelenHunt_44_f.jpg,44,41-50
4,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10047_HelenHunt_34_f.jpg,34,31-40
...,...,...,...,...,...,...,...,...,...
523,566,ElkeSommer,0,9971_ElkeSommer_62_f.jpg,62,61-70,9960_ElkeSommer_42_f.jpg,42,41-50
524,566,ElkeSommer,0,9971_ElkeSommer_62_f.jpg,62,61-70,9946_ElkeSommer_18_f.jpg,18,11-20
525,566,ElkeSommer,0,9973_ElkeSommer_64_f.jpg,64,61-70,9960_ElkeSommer_42_f.jpg,42,41-50
526,566,ElkeSommer,0,9973_ElkeSommer_64_f.jpg,64,61-70,9946_ElkeSommer_18_f.jpg,18,11-20


In [4]:
CASIA_df

,ID,Gender,First Image,First Age,First Age Range,Second Image,Second Age,Second Age Range
0,2,1,000002_00000273.jpg,44.5,41-50,000002_00000274.jpg,46.0,41-50
1,2,1,000002_00000273.jpg,44.5,41-50,000002_00000275.jpg,56.0,51-60
2,2,1,000002_00000273.jpg,44.5,41-50,000002_00000276.jpg,31.5,31-40
3,2,1,000002_00000273.jpg,44.5,41-50,000002_00000277.jpg,34.5,31-40
4,2,1,000002_00000273.jpg,44.5,41-50,000002_00000279.jpg,46.0,41-50
...,...,...,...,...,...,...,...,...
523,10553,1,010553_00490113.jpg,34.0,31-40,010553_00490115.jpg,26.0,21-30
524,10553,1,010553_00490113.jpg,34.0,31-40,010553_00490116.jpg,36.5,31-40
525,10553,1,010553_00490114.jpg,34.5,31-40,010553_00490115.jpg,26.0,21-30
526,10553,1,010553_00490114.jpg,34.5,31-40,010553_00490116.jpg,36.5,31-40


In [5]:
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

def face_similarities(paths):

    first_image = ins_get_image(paths[0])
    second_image = ins_get_image(paths[1])

    first_face = app.get(first_image)
    second_face = app.get(second_image)

    for face in first_face:
        first_feature = face.normed_embedding

    for face in second_face:
        second_feature = face.normed_embedding

    first_feature_np = np.array(first_feature)
    second_feature_np = np.array(second_feature)

    # Cosine similarites
    sims = np.dot(first_feature_np, second_feature_np.T) / (np.linalg.norm(first_feature_np) * np.linalg.norm(second_feature_np))

    return first_feature_np, second_feature_np, sims

In [6]:
from multiprocessing import Pool

first_filename_list_AgeDB = AgeDB_df["First Image"].to_list()
second_filename_list_AgeDB = AgeDB_df["Second Image"].to_list()

AgeDB_filename_pairs = []

for i in range(len(first_filename_list_AgeDB)):
    first_elem = '/home/ivarbl/FaceProject/FaceProject/Images/AgeDB/' + str(first_filename_list_AgeDB[i][:-4])
    second_elem = '/home/ivarbl/FaceProject/FaceProject/Images/AgeDB/' + str(second_filename_list_AgeDB[i][:-4])
    AgeDB_filename_pairs.append((first_elem,second_elem))

app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(256,256))

AgeDB_sims_output = []

with Pool() as pool:
    for output in pool.map(face_similarities, AgeDB_filename_pairs):
        AgeDB_sims_output.append(output)


/home/ivarbl/FaceEnvUU/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ivarbl/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ivarbl/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ivarbl/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ivarbl/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ivarbl/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
se

/home/ivarbl/FaceEnvUU/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
/home/ivarbl/FaceEnvUU/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
/home/ivarbl/FaceEnvUU/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the d

In [7]:
First_Image_Features = []
Second_Image_Features = []
Cosine_Similarity = []

for tripple in AgeDB_sims_output:
    First_Image_Features.append(tripple[0])
    Second_Image_Features.append(tripple[1])
    Cosine_Similarity.append(tripple[2])

AgeDB_df["First Image Features"] = First_Image_Features
AgeDB_df["Second Image Features"] = Second_Image_Features
AgeDB_df["Cosine Similarity"] = Cosine_Similarity

In [8]:
AgeDB_df

,ID,Name,Gender,First Image,First Age,First Age Range,Second Image,Second Age,Second Age Range,First Image Features,Second Image Features,Cosine Similarity
0,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10044_HelenHunt_32_f.jpg,32,31-40,"[-0.024523553, -0.049861606, 0.10224585, -0.00...","[-0.02340692, -0.034321308, 0.055801727, 0.004...",0.637691
1,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10045_HelenHunt_33_f.jpg,33,31-40,"[-0.024523553, -0.049861606, 0.10224585, -0.00...","[-0.028526206, -0.019948885, 0.02602013, -0.03...",0.615305
2,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10046_HelenHunt_34_f.jpg,34,31-40,"[-0.024523553, -0.049861606, 0.10224585, -0.00...","[-0.019059857, 0.0010646223, 0.04850586, -0.08...",0.713908
3,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10057_HelenHunt_44_f.jpg,44,41-50,"[-0.024523553, -0.049861606, 0.10224585, -0.00...","[-0.037811007, -0.051191434, 0.08856339, -0.02...",0.887338
4,3,HelenHunt,0,10058_HelenHunt_45_f.jpg,45,41-50,10047_HelenHunt_34_f.jpg,34,31-40,"[-0.024523553, -0.049861606, 0.10224585, -0.00...","[-0.04501873, -0.013367287, 0.03943252, -0.081...",0.572509
...,...,...,...,...,...,...,...,...,...,...,...,...
523,566,ElkeSommer,0,9971_ElkeSommer_62_f.jpg,62,61-70,9960_ElkeSommer_42_f.jpg,42,41-50,"[0.023328878, -0.09159723, -0.02289407, -0.014...","[0.04827074, 0.009286343, -0.01057999, 0.03953...",0.467892
524,566,ElkeSommer,0,9971_ElkeSommer_62_f.jpg,62,61-70,9946_ElkeSommer_18_f.jpg,18,11-20,"[0.023328878, -0.09159723, -0.02289407, -0.014...","[-0.047902714, 0.039434005, 0.067246035, -0.05...",0.170420
525,566,ElkeSommer,0,9973_ElkeSommer_64_f.jpg,64,61-70,9960_ElkeSommer_42_f.jpg,42,41-50,"[0.003156112, -0.059776146, -0.047906604, 0.02...","[0.04827074, 0.009286343, -0.01057999, 0.03953...",0.481166
526,566,ElkeSommer,0,9973_ElkeSommer_64_f.jpg,64,61-70,9946_ElkeSommer_18_f.jpg,18,11-20,"[0.003156112, -0.059776146, -0.047906604, 0.02...","[-0.047902714, 0.039434005, 0.067246035, -0.05...",0.248497


In [9]:
# Saving work
AgeDB_df.to_csv("createdCSV/AgeDB_result.csv")

In [16]:
from multiprocessing import Pool

first_filename_list_CASIA = CASIA_df["First Image"].to_list()
second_filename_list_CASIA = CASIA_df["Second Image"].to_list()

CASIA_filename_pairs = []

for i in range(len(first_filename_list_CASIA)):
    first_elem = '/home/ivarbl/FaceProject/FaceProject/Images/casia-webface-valid-remote/' + str(first_filename_list_CASIA[i][:-4])
    second_elem = '/home/ivarbl/FaceProject/FaceProject/Images/casia-webface-valid-remote/' + str(second_filename_list_CASIA[i][:-4])
    CASIA_filename_pairs.append((first_elem,second_elem))



app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(128,128))

CASIA_sims_output = []

with Pool() as pool:
    for output in pool.map(face_similarities, CASIA_filename_pairs):
        CASIA_sims_output.append(output)


/home/ivarbl/FaceEnvUU/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ivarbl/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ivarbl/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ivarbl/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ivarbl/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ivarbl/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
se

/home/ivarbl/FaceEnvUU/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
/home/ivarbl/FaceEnvUU/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
/home/ivarbl/FaceEnvUU/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the d

In [17]:
First_Image_Features_CASIA = []
Second_Image_Features_CASIA = []
Cosine_Similarity_CASIA = []

for tripple in CASIA_sims_output:
    First_Image_Features_CASIA.append(tripple[0])
    Second_Image_Features_CASIA.append(tripple[1])
    Cosine_Similarity_CASIA.append(tripple[2])

CASIA_df["First Image Features"] = First_Image_Features_CASIA
CASIA_df["Second Image Features"] = Second_Image_Features_CASIA
CASIA_df["Cosine Similarity"] = Cosine_Similarity_CASIA

In [18]:
CASIA_df

,ID,Gender,First Image,First Age,First Age Range,Second Image,Second Age,Second Age Range,First Image Features,Second Image Features,Cosine Similarity
0,2,1,000002_00000273.jpg,44.5,41-50,000002_00000274.jpg,46.0,41-50,"[0.04148592, 0.046524275, 0.004078803, -0.0510...","[0.064954944, -0.010699481, -0.027605478, 0.04...",0.528167
1,2,1,000002_00000273.jpg,44.5,41-50,000002_00000275.jpg,56.0,51-60,"[0.04148592, 0.046524275, 0.004078803, -0.0510...","[0.029864999, 0.024729883, 0.062448174, -0.064...",0.579516
2,2,1,000002_00000273.jpg,44.5,41-50,000002_00000276.jpg,31.5,31-40,"[0.04148592, 0.046524275, 0.004078803, -0.0510...","[-0.05003019, -0.040564433, 0.09071848, -0.004...",0.500336
3,2,1,000002_00000273.jpg,44.5,41-50,000002_00000277.jpg,34.5,31-40,"[0.04148592, 0.046524275, 0.004078803, -0.0510...","[-0.052534938, -0.08337941, 0.08544136, 0.0372...",0.397464
4,2,1,000002_00000273.jpg,44.5,41-50,000002_00000279.jpg,46.0,41-50,"[0.04148592, 0.046524275, 0.004078803, -0.0510...","[-0.09699498, -0.029979127, 0.06556815, -0.011...",0.645424
...,...,...,...,...,...,...,...,...,...,...,...
523,10553,1,010553_00490113.jpg,34.0,31-40,010553_00490115.jpg,26.0,21-30,"[0.06784377, 0.07660285, -0.022913313, 0.01612...","[0.0063402066, 0.002414208, -0.027150378, -0.0...",0.527963
524,10553,1,010553_00490113.jpg,34.0,31-40,010553_00490116.jpg,36.5,31-40,"[0.06784377, 0.07660285, -0.022913313, 0.01612...","[0.03470395, 0.052519906, -0.0065446934, 0.077...",0.595495
525,10553,1,010553_00490114.jpg,34.5,31-40,010553_00490115.jpg,26.0,21-30,"[0.037600536, 0.044978686, 0.021523183, 0.0268...","[0.0063402066, 0.002414208, -0.027150378, -0.0...",0.509435
526,10553,1,010553_00490114.jpg,34.5,31-40,010553_00490116.jpg,36.5,31-40,"[0.037600536, 0.044978686, 0.021523183, 0.0268...","[0.03470395, 0.052519906, -0.0065446934, 0.077...",0.602436


In [20]:
CASIA_df.to_csv("/proj/snic2021-22-918/CASIA_result.csv")